### Preperation

In [1]:
!pip install torchviz

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# PyTorch modules
import torch
import torch.nn as nn
import torch.optim as optim # optim is a package implementing optimization algorithms
import torch.nn.functional as F # nn.functional contains some loss functions and other utilities
from torch.utils.data import Dataset, DataLoader # dataloader is used for dividing dataset into batches

from torchviz import make_dot, make_dot_from_trace # for visualizing computational graph
from tqdm.notebook import tqdm # for displaying progress bar


In [3]:
train=pd.read_csv('train.csv',index_col='id')
test=pd.read_csv('test.csv',index_col='id')

TARGET='target'
RANDOM_STATE=2021

In [4]:
# check for duplicated rows
print(f"Duplicated rows: {train.drop(TARGET,axis=1).duplicated().sum()}")
# drop duplicate rows
train=train[~train.drop('target',axis=1).duplicated()]
train.shape

Duplicated rows: 4


(99996, 51)

In [5]:
# train data summary
train.describe().T


,count,mean,std,min,25%,50%,75%,max
feature_0,99996.0,0.257840,0.929050,0.0,0.0,0.0,0.0,10.0
feature_1,99996.0,0.431737,1.977899,0.0,0.0,0.0,0.0,31.0
feature_2,99996.0,0.114135,0.519593,0.0,0.0,0.0,0.0,6.0
feature_3,99996.0,0.590574,1.844591,0.0,0.0,0.0,0.0,26.0
feature_4,99996.0,0.599794,2.785584,0.0,0.0,0.0,0.0,38.0
feature_5,99996.0,0.160826,0.601161,0.0,0.0,0.0,0.0,10.0
feature_6,99996.0,0.731519,2.343507,0.0,0.0,0.0,0.0,27.0
feature_7,99996.0,1.228959,2.692778,0.0,0.0,0.0,1.0,31.0
feature_8,99996.0,0.903386,3.415321,0.0,0.0,0.0,0.0,39.0
feature_9,99996.0,0.940158,1.904201,0.0,0.0,0.0,1.0,17.0


In [7]:
X=train.drop(columns=[TARGET],axis=1)
X.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,1,0,0,0,0,0,...,3,0,0,21,0,0,0,0,0,0
1,0,0,0,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,1,0,0,0,0,13,2,0
3,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
X.shape

(99996, 50)